In [40]:
import pandas as pd
import numpy as np
import sklearn as sl
from IPython.display import display

In [41]:
df = pd.read_csv('cosmetics.csv')

df.columns = df.columns.str.lower().str.strip()



In [42]:
df.head()

,label,brand,name,price,rank,ingredients,combination,dry,normal,oily,sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [43]:
df['label'].unique()

array(['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream',
       'Sun protect'], dtype=object)

In [44]:

# --- Define Ingredient-Problem Mapping ---
# This is a dictionary where keys are skin problems and values are lists of ingredient names
# that are known to help with that problem. You will need to expand this list
# based on your knowledge or research about skincare ingredients.
ingredient_problem_map = {
    'acne': ['salicylic acid', 'benzoyl peroxide', 'niacinamide', 'tea tree oil'],
    'dryness': ['hyaluronic acid', 'glycerin', 'ceramides', 'squalane', 'shea butter'],
    'redness': ['niacinamide', 'centella asiatica', 'licorice extract', 'chamomile'],
    'aging': ['retinol', 'vitamin c', 'peptides', 'hyaluronic acid', 'niacinamide'],
    'dark spots': ['vitamin c', 'niacinamide', 'alpha arbutin', 'kojic acid'],
    'oiliness': ['niacinamide', 'salicylic acid', 'clay', 'zinc']
    # Add more problems and ingredients as needed
}

# Function to recommend products based on user input including problems
def recommend_products_from_input(product_types=None, brands=None, skin_types=None, problems=None, max_price=100, top_n=5):
    filtered_df = df.copy()

    # Filter by product type if given
    if product_types:
        filtered_df = filtered_df[filtered_df['label'].isin(product_types)]

    # Filter by brand if given
    if brands:
        filtered_df = filtered_df[filtered_df['brand'].isin(brands)]

    # Filter by skin type if given (assuming skin types are in separate columns with 1/0)
    if skin_types:
        skin_type_filters = []
        for skin_type in skin_types:
            if skin_type in filtered_df.columns:
                skin_type_filters.append(filtered_df[skin_type] == 1)
            else:
                print(f"Warning: Skin type column '{skin_type}' not found in the data.")

        if skin_type_filters:
            combined_skin_filter = skin_type_filters[0]
            for i in range(1, len(skin_type_filters)):
                combined_skin_filter = combined_skin_filter | skin_type_filters[i]
            filtered_df = filtered_df[combined_skin_filter]
        else:
             pass # No valid skin type filters means no skin type filtering applied

    # Filter by problems (ingredients) if given
    if problems:
        # Get the list of relevant ingredients for the selected problems
        relevant_ingredients = []
        for problem in problems:
            if problem.lower() in ingredient_problem_map:
                relevant_ingredients.extend(ingredient_problem_map[problem.lower()])
            else:
                print(f"Warning: Information for problem '{problem}' not found in the ingredient map.")

        # Remove duplicates and convert to lowercase for case-insensitive matching
        relevant_ingredients = list(set([ing.lower() for ing in relevant_ingredients]))

        if relevant_ingredients:
            # Filter products that contain at least one of the relevant ingredients
            # We assume the 'ingredients' column contains a string of ingredients,
            # typically separated by commas or other delimiters. We'll check if
            # any of the relevant ingredients are present in the ingredient string.
            ingredient_filter = filtered_df['ingredients'].apply(
                lambda ingredient_list: any(ing in ingredient_list.lower() for ing in relevant_ingredients) if pd.notna(ingredient_list) else False
            )
            filtered_df = filtered_df[ingredient_filter]
        else:
            print("No relevant ingredients found for the specified problems.")
            # Optionally, you might want to return an empty DataFrame here
            # if no ingredients match the problems, depending on desired behavior.
            # filtered_df = filtered_df.head(0) # This would ensure no recommendations if no ingredients are found

    # Filter by price
    filtered_df = filtered_df[filtered_df['price'] <= max_price]

    # Sort by rank (lower rank = better)
    filtered_df = filtered_df.sort_values(by='rank')

    # Select top N
    return filtered_df[['brand', 'name', 'label', 'price', 'rank', 'ingredients']].head(top_n)

## Get user input

# Get product type input
while True:
    product_type_input = input("Enter product types (e.g., Moisturizer, Cleanser, Treatment, Face Mask, Eye cream, Sun protect ), (( separate with commas)), or press Enter to skip: ").strip()
    if not product_type_input:
        user_product_types = None
        break
    user_product_types = [item.strip() for item in product_type_input.split(',')]
    # Optional: Add validation here to check if entered product types exist in the data
    break

# Get brand input
while True:
    brand_input = input("Enter brands ( separate with commas), or press Enter to skip: ").strip()
    if not brand_input:
        user_brands = None
        break
    user_brands = [item.strip() for item in brand_input.split(',')]
    # Optional: Add validation here to check if entered brands exist in the data
    break

# Get skin type input (assuming skin types are in separate columns with 1/0)
while True:
    skin_type_input = input("Enter skin types (e.g : oily, sensitive, dry, normal, combination, separate with commas), or press Enter to skip: ").strip()
    if not skin_type_input:
        user_skin_types = None
        break
    user_skin_types = [item.strip() for item in skin_type_input.split(',')]
    # Optional: Add validation here to check if entered skin types are valid column names
    break

# --- Get user problems input ---
while True:
    problems_input = input(f"Enter skin problems (e.g., acne, dryness, redness, separate with commas), or press Enter to skip: ").strip()
    if not problems_input:
        user_problems = None
        break
    user_problems = [item.strip() for item in problems_input.split(',')]
    # Optional: Add validation here to check if entered problems are valid
    break


# Get max price input
while True:
    max_price_input = input(f"Enter maximum price (default is 100), or press Enter to use default: ").strip()
    if not max_price_input:
        user_max_price = 100
        break
    try:
        user_max_price = float(max_price_input)
        if user_max_price < 0:
             print("Price cannot be negative. Please enter a valid price.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a number for the price.")

# Get number of recommendations input
while True:
    top_n_input = input(f"Enter the number of recommendations (default is 5), or press Enter to use default: ").strip()
    if not top_n_input:
        user_top_n = 5
        break
    try:
        user_top_n = int(top_n_input)
        if user_top_n <= 0:
             print("Number of recommendations must be positive. Please enter a valid number.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter an integer for the number of recommendations.")

## Get recommendations and display

print("\nGenerating recommendations...")
recommendations = recommend_products_from_input(
    product_types=user_product_types,
    brands=user_brands,
    skin_types=user_skin_types,
    problems=user_problems, # Pass the user's problems to the function
    max_price=user_max_price,
    top_n=user_top_n
)

if recommendations.empty:
    print("No products found matching your criteria.")
else:
    print("\nRecommended Products:")
    display(recommendations) # Use display for better formatting in Jupyter


Enter product types (e.g., Moisturizer, Cleanser, Treatment, Face Mask, Eye cream, Sun protect ), (( separate with commas)), or press Enter to skip: Moisturizer
Enter brands ( separate with commas), or press Enter to skip: 
Enter skin types (e.g : oily, sensitive, dry, normal, combination, separate with commas), or press Enter to skip: combination
Enter skin problems (e.g., acne, dryness, redness, separate with commas), or press Enter to skip: acne
Enter maximum price (default is 100), or press Enter to use default: 
Enter the number of recommendations (default is 5), or press Enter to use default: 

Generating recommendations...

Recommended Products:


,brand,name,label,price,rank,ingredients
87,IT COSMETICS,Secret Sauce Clinically Advanced Miraculous An...,Moisturizer,68,3.8,"Water, Butylene Glycol, Cetyl Ethylhexanoate, ..."
34,IT COSMETICS,Your Skin But Better CC+ Cream Oil-Free Matte ...,Moisturizer,38,3.9,"Water, Dimethicone, Butylene Glycol Dicaprylat..."
70,IT COSMETICS,Your Skin But Better™ CC+Illumination™ Cream w...,Moisturizer,38,3.9,"Water, Snail Secretion Filtrate, Titanium Diox..."
4,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,Moisturizer,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet..."
143,SMASHBOX,Photo Finish Primerizer,Moisturizer,42,4.1,"Water, Glycerin, Dimethicone, Isopropyl Isoste..."
